In [1]:
import numpy as np 
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go
import mysql.connector
import pandas as pd
from plotly.graph_objs import *
import datetime 
import time

In [2]:
# connect to plotly
import plotly
plotly.tools.set_credentials_file(username='username', api_key='123456789fake')

In [3]:
stream_ids = ['yqqxxi6xv7', 'i6sj2mp6ao']
print(stream_ids)

['yqqxxi6xv7', 'i6sj2mp6ao']


In [4]:
# Get stream id from stream id list 
stream_id = stream_ids[0]

# Make instance of stream id object 
stream_1 = go.Stream(
    token=stream_id,  # link stream id to 'token' key
    maxpoints=80      # keep a max of 80 pts on screen
)

C:\Users\Cynthia\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:504: DeprecationWarning:

plotly.graph_objs.Stream is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Stream
  - plotly.graph_objs.area.Stream




In [5]:
# Initialize trace of streaming plot by embedding the unique stream_id
trace1 = go.Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    stream=stream_1         #  1 per trace
)

data = go.Data([trace1])

C:\Users\Cynthia\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [6]:
# Add title to layout object
layout = go.Layout(title='Temperature Time Series')

# Make a figure object
fig = go.Figure(data=data, layout=layout)

# Send fig to Plotly, initialize streaming plot, open new tab
py.iplot(fig, filename='python-streaming')

In [7]:
# provide the stream link object with the same token associated with the trace to stream
s = py.Stream(stream_id)

# open a connection
s.open()

In [8]:
#connect to the database
config = {
  'user': 'root',
  'password': '',
  'host': 'localhost',
  'db': 'labproject', 
}

# loop for updating the measurement to plot
while True:
    # get value from database
    db = mysql.connector.connect(**config)
    cursor = db.cursor()
    temps = "SELECT * FROM temperature"
    cursor.execute(temps)
    rows = cursor.fetchall()
    df = pd.DataFrame( [[ij for ij in i] for i in rows] )
    df.rename(columns={0: 'id', 1: 'date', 2: 'temp'}, inplace=True)
    
    # Current time on x-axis, random numbers on y-axis
    x = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
    y = df.temp.iloc[-1] 
        
    # Send data to plot
    s.write(dict(x=x, y=y))  
    
    time.sleep(10)  # plot a point every 10 seconds   
    
# Close the stream & db when done plotting
s.close()
cursor.close()
db.close()

KeyboardInterrupt: 